In [2]:
!pip install transformers datasets peft
import os
import zipfile
import requests

url = "https://cims.nyu.edu/~sbowman/multinli/multinli_1.0.zip"
r = requests.get(url)
with open("multinli_1.0.zip", "wb") as f:
    f.write(r.content)

with zipfile.ZipFile("multinli_1.0.zip", "r") as zip_ref:
    zip_ref.extractall()

os.listdir("multinli_1.0")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 16.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.

['multinli_1.0_train.txt',
 'Icon\r',
 'multinli_1.0_train.jsonl',
 'multinli_1.0_dev_matched.jsonl',
 'README.txt',
 'multinli_1.0_dev_mismatched.txt',
 'multinli_1.0_dev_mismatched.jsonl',
 'paper.pdf',
 '.DS_Store',
 'multinli_1.0_dev_matched.txt']

In [3]:
from transformers import RobertaForSequenceClassification, RobertaTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict, Dataset
import json
import pandas as pd

def load_multinli_data(split):
    if split == 'train':
        file_path = 'multinli_1.0/multinli_1.0_train.jsonl'
    elif split == 'validation':
        file_path = 'multinli_1.0/multinli_1.0_dev_matched.jsonl'
    else:
        raise ValueError("Invalid split. Use 'train' or 'validation'")

    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))

    return data

def create_dataset_dict():
    train_data = load_multinli_data('train')
    validation_data = load_multinli_data('validation')
    train_df = pd.DataFrame(train_data)
    validation_df = pd.DataFrame(validation_data)
    train_df = train_df.rename(columns={"premise": "sentence1", "hypothesis": "sentence2", "gold_label": "label"})
    validation_df = validation_df.rename(columns={"premise": "sentence1", "hypothesis": "sentence2", "gold_label": "label"})
    train_df = train_df[train_df['label'].isin(['entailment', 'neutral', 'contradiction'])]
    validation_df = validation_df[validation_df['label'].isin(['entailment', 'neutral', 'contradiction'])]
    label_mapping = {'entailment': 0, 'neutral': 1, 'contradiction': 2}
    train_df['label'] = train_df['label'].map(label_mapping)
    validation_df['label'] = validation_df['label'].map(label_mapping)
    return DatasetDict({
        'train': Dataset.from_pandas(train_df),
        'validation': Dataset.from_pandas(validation_df)
    })

dataset = create_dataset_dict()
model_name = 'roberta-large'
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = RobertaTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples['sentence1'], examples['sentence2'], truncation='longest_first', padding='max_length', max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
columns_to_keep = ['input_ids', 'attention_mask', 'label']
tokenized_dataset = tokenized_dataset.remove_columns(set(tokenized_dataset['train'].column_names) - set(columns_to_keep))
tokenized_dataset.set_format("torch")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/392702 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [13]:
from datasets import load_metric
accuracy_metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=3,
    weight_decay=0.01,
    gradient_accumulation_steps=2,
    fp16=True,
    dataloader_num_workers=8,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'].shuffle().select(range(int(0.1 * len(tokenized_dataset['train'])))),
    eval_dataset=tokenized_dataset['validation'],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate()
print(results)

/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLo

Epoch,Training Loss,Validation Loss,Accuracy
0,0.405800,0.354279,0.874885
2,0.137000,0.543746,0.881915


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: Runtime

{'eval_loss': 0.5437459349632263, 'eval_accuracy': 0.8819154355578197, 'eval_runtime': 68.7286, 'eval_samples_per_second': 142.808, 'eval_steps_per_second': 11.902, 'epoch': 2.999083409715857}


In [14]:
print(f"Accuracy: {results['eval_accuracy']}")

Accuracy: 0.8819154355578197
